In [1]:
import pandas as pd

In [2]:
hdf = pd.HDFStore("run592_hh_job_mon_refined_popfix.h5", complib='zlib', complevel=5)

In [3]:
hdf

<class 'pandas.io.pytables.HDFStore'>
File path: run592_hh_job_mon_refined_popfix.h5
/2020/buildings                                             frame        (shape->[1687933,13])
/2020/dropped_buildings                                     frame        (shape->[9159,14])   
/2020/group_quarters                                        frame        (shape->[82171,5])   
/2020/households                                            frame        (shape->[1,8])       
/2020/jobs                                                  frame        (shape->[2867718,17])
/2020/parcels                                               frame        (shape->[1794429,15])
/2020/persons                                               frame        (shape->[4688989,7]) 
/2025/buildings                                             frame        (shape->[1691438,13])
/2025/dropped_buildings                                     frame        (shape->[9847,14])   
/2025/group_quarters                                        

In [5]:
refinement = pd.read_csv("MonForecastHHPOPRefinement.csv").set_index('b_city_id')

In [6]:
refinement

,2020,2025,2030,2035,2040,2045
b_city_id,,,,,,
5015,-300,-300,-300,-300,-300,-300
5020,-75,-75,-75,-75,-75,-75
5025,-250,-250,-275,-275,-275,-275
5030,100,100,100,100,100,100
5035,40,40,-60,-60,-60,-60
5040,-15,-5,20,-10,-30,-10
5045,100,100,100,100,100,100
5050,0,0,0,-70,-70,-70
5065,0,-35,-35,-35,-35,-35


In [7]:
refinement.sum()

2020    0
2025    0
2030    0
2035    0
2040    0
2045    0
dtype: int64

In [8]:
refinement.columns.sort_values().unique()

Index([u'2020', u'2025', u'2030', u'2035', u'2040', u'2045'], dtype='object')

In [9]:
for year in refinement.columns.sort_values().unique():
    print "year", year
    b_city_id = hdf[str(year) + '/buildings'][['b_city_id']]
    households = hdf[str(year) + '/households']
    households_col = households.columns
    households = pd.merge(households, b_city_id, left_on='building_id', right_index=True, how='left')
    pre = households[households.b_city_id.isin(refinement.index)].groupby('b_city_id').persons.sum()
    ref = refinement[year]
    gole = pre + ref
    while len(ref) > 0 and ref.abs().sum() > 0:
        ref = ref[ref != 0].copy()
        main_city = ref.abs().argmin()
        target = ref.loc[main_city]
        other_citys = ref[ref * target < 0]

        hh_main = households[households.b_city_id == main_city]
        hh_other = households[households.b_city_id.isin(other_citys.index)]
        
        sample_size = min(10 * abs(target), len(hh_main), len(hh_other))
        
        print year, main_city, target, len(hh_main), len(hh_other), sample_size

        hh_main_sample = hh_main.sample(sample_size)
        hh_other_sample = hh_other.sample(sample_size)
        
        corect_dir = (target * (hh_other_sample.persons.values - hh_main_sample.persons.values) > 0)
        hh_main_sample = hh_main_sample[corect_dir]
        hh_other_sample = hh_other_sample[corect_dir]
        
        corect_size = (abs(hh_other_sample.persons.values - hh_main_sample.persons.values) <= abs(target))
        hh_main_sample = hh_main_sample[corect_size]
        hh_other_sample = hh_other_sample[corect_size]
        
        fit = abs(hh_other_sample.persons.values - hh_main_sample.persons.values).cumsum() <= abs(target)
        hh_main_sample = hh_main_sample[fit]
        hh_other_sample = hh_other_sample[fit]

        households.loc[hh_main_sample.index, 'building_id'] = hh_other_sample.building_id.values
        households.loc[hh_main_sample.index, 'b_city_id'] = hh_other_sample.b_city_id.values
        households.loc[hh_other_sample.index, 'building_id'] = hh_main_sample.building_id.values
        households.loc[hh_other_sample.index, 'b_city_id'] = hh_main_sample.b_city_id.values
        
        post = households[households.b_city_id.isin(refinement.index)].groupby('b_city_id').persons.sum()
        
        ref = (gole - post).astype(int)
    
    hdf[str(year) + '/households'] = households[households_col]
    print "done"

year 2020
2020 5040 -15 196 18270 150
2020 5040 -1 196 18270 10
2020 5040 -1 196 18270 10
2020 5035 35 1814 6525 350
2020 5120 47 1698 6525 470
2020 5120 1 1698 6525 10
2020 5020 -66 975 14758 660
2020 5090 46 8073 5550 460
2020 5090 2 8073 5550 20
2020 5070 -67 684 6685 670
2020 5070 -2 684 6685 20
2020 5105 61 2218 4866 610
2020 5085 65 637 4866 637
2020 5100 69 407 4866 407
2020 5045 73 1260 4866 730
2020 5015 -48 3086 2163 480
2020 5030 56 1039 1780 560
2020 5030 1 1039 1780 10
2020 5030 1 1039 1780 10
2020 5025 -67 1780 1124 670
2020 5025 -1 1780 1124 10


/home/da/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2885: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block0_values] [items->['age_of_head', 'cars', 'workers', 'race_id', 'children']]

  exec(code_obj, self.user_global_ns, self.user_ns)


year 2025
2025 5040 -5 214 17616 50
2025 5070 -25 712 17616 250
2025 5070 -1 712 17616 10
2025 5100 20 509 7380 200
2025 5100 1 509 7380 10
2025 5065 -33 1222 17107 330
2025 5065 -1 1222 17107 10
2025 5035 31 1848 6158 310
2025 5120 43 1708 6158 430
2025 5020 -70 990 13551 700
2025 5085 70 692 5168 692
2025 5085 3 692 5168 30
2025 5090 90 8223 5168 900
2025 5090 2 8223 5168 20
2025 5090 1 8223 5168 10
2025 5030 96 1058 5168 960
2025 5030 2 1058 5168 20
2025 5030 1 1058 5168 10
2025 5015 -56 3334 3578 560
2025 5015 -2 3334 3578 20
2025 5045 69 1308 1834 690
2025 5025 -82 1834 2270 820


/home/da/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2885: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block1_values] [items->['cars', 'workers', 'race_id', 'age_of_head', 'children']]

  exec(code_obj, self.user_global_ns, self.user_ns)


year 2030
2030 5040 20 208 9570 200
2030 5040 2 208 9570 20
2030 5070 25 712 9570 250
2030 5065 -24 1232 16143 240
2030 5100 24 552 8338 240
2030 5035 -45 1930 15591 450
2030 5035 -3 1930 15591 30
2030 5120 42 1699 6408 420
2030 5020 -61 1033 13892 610
2030 5020 -2 1033 13892 20
2030 5085 66 701 5375 660
2030 5085 3 701 5375 30
2030 5030 91 1108 5375 910
2030 5090 91 8415 5375 910
2030 5015 -76 3479 3668 760
2030 5015 -1 3479 3668 10
2030 5045 56 1356 1896 560
2030 5025 -124 1896 2312 1240
2030 5025 -2 1896 2312 20
year 2035
2035 5040 -10 205 17044 100
2035 5070 25 721 18686 250
2035 5100 25 617 18686 250
2035 5065 -28 1269 15706 280
2035 5120 50 1719 17417 500
2035 5120 1 1719 17417 10
2035 5050 -28 8733 13987 280
2035 5050 -3 8733 13987 30
2035 5035 -50 1978 13987 500
2035 5020 -69 1067 13987 690
2035 5090 64 8462 5639 640
2035 5090 1 8462 5639 10
2035 5085 70 718 5639 700
2035 5085 2 718 5639 20
2035 5045 82 1357 5639 820
2035 5030 90 1128 5639 900
2035 5030 1 1128 5639 10
2035 5015

In [10]:
hdf.close()